In [ ]:
from keras import optimizers
from keras.layers.recurrent import LSTM
from keras.layers import Reshape, Lambda, BatchNormalization
import keras_preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
train_dir=os.path.join('./train_under/')
val_dir=os.path.join('./val/')
test_dir=os.path.join('./test/')
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(train_dir,
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator=train_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='categorical')
val_datagen=ImageDataGenerator(rescale=1/255)
val_generator=train_datagen.flow_from_directory(val_dir,
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:
#model
#Resnet model
input = Input(shape=(224, 224, 3))
model=ResNet50(
    weights="imagenet",
    input_tensor=input,
    pooling='max',
    include_top=False,
)
x = model.output
x = Dense(1024, name='fully', kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512, kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(128)(x)
x = Reshape((1,128))(x)
x = LSTM(128, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(x)
x = Dense(2, kernel_initializer='he_normal',name='dense1')(x)
x = Activation('softmax', name='softmax')(x)
model = Model(model.input, x)
model.summary()
sgd = optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
            loss='binary_crossentropy')

In [ ]:
def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * 0.1

In [ ]:
from typing import List, Tuple
from keras.callbacks import Callback
from sklearn.metrics import classification_report
import numpy as np
import itertools

class Metrics(Callback):

    def __init__(self, val_data,batch_size: int = 32):
        super().__init__()
        self.validation_data = val_data
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs={}):
        val_true, val_pred = evaluate(data_gen=self.validation_data, model=self.model)
        val_true_class = np.argmax(val_true, axis=1)
        val_pred_class = np.argmax(val_pred, axis=1)

        cls_report = classification_report(
            y_true=val_true_class,
            y_pred=val_pred_class,
            output_dict=True,
            target_names=['0','1'],
            labels=np.arange(2)        )
        with open('./weights/logs','a') as f:
            f.write(str(epoch)+'         '+str(cls_report)+'\n')
        print(cls_report)



def evaluate(data_gen, model) -> Tuple[np.ndarray]:
    """
    Uses a model to predict on all examples in a data generator.

    Args:
        data_gen: Keras data generator
        model: Keras model
        n_batches: int

    Returns:
        y_true: Ground truth labels. Shape (n_examples, )
        y_pred: Predicted labels. Shape (n_examples, )

    """
    y_preds = []
    y_trues = []
    for idx in range(len(data_gen)):
        print('======================{}======================\n'.format(idx))
        X, y_true = next(data_gen)
        y_trues.append(y_true)
        y_pred = model.predict(X)
        y_preds.extend(y_pred)

    y_true = np.concatenate(y_trues, axis=0)
    y_pred = np.concatenate(y_preds, axis=0)

    return y_true, y_pred


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

with tf.device('/gpu:0'):
    filename = './weights/weights-improvement-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(filename,             
                                 monitor='val_loss',   
                                 verbose=1,           
                                 save_best_only=False,
                                 mode='auto' 
                                )
    #training
    history = model.fit_generator(train_generator,
                                  epochs=60,
                                  steps_per_epoch=len(train_generator),
                                  validation_data=val_generator,
                                  validation_steps=1,
                                  callbacks=[Metrics(val_generator),
                                            tf.keras.callbacks.LearningRateScheduler(scheduler),
                                            checkpoint]
                                 )

In [ ]:
from keras.models import load_model
model = load_model('./weights/weights-improvement-14.hdf5')


In [ ]:
def test_evaluate(data_gen, model) -> Tuple[np.ndarray]:

    y_preds = []
    y_trues = []
    for idx in range(len(data_gen)):
        print('======================{}======================\n'.format(idx))
        X, y_true = next(data_gen)
        y_trues.append(y_true)
        y_pred = model.predict(X)
        y_preds.extend(y_pred)

    y_true = np.concatenate(y_trues, axis=0)
    y_pred = np.concatenate(y_preds, axis=0)
    
    val_true, val_pred = y_true,y_pred
    val_true_class = np.argmax(val_true, axis=1)
    val_pred_class = np.argmax(val_pred, axis=1)

    cls_report = classification_report(
        y_true=val_true_class,
        y_pred=val_pred_class,
        output_dict=True,
        target_names=['0','1'],
        labels=np.arange(2)        )

    print(cls_report)

    return y_true, y_pred

In [ ]:
test_evaluate(test_generator,model)

In [ ]:
#feature_extract
train_dir=os.path.join('./train/')
val_dir=os.path.join('./val/')
test_dir=os.path.join('./test/')
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(train_dir,
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator=train_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='categorical')
val_datagen=ImageDataGenerator(rescale=1/255)
val_generator=train_datagen.flow_from_directory(val_dir,
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:
#feature extract
fe_model = Model(inputs=model.input, outputs=model.get_layer('lstm1').output)


In [ ]:
import json
from tensorflow.keras.preprocessing import image
import numpy as np
with open('./train_val_test','r') as f:
    data_split=json.load(f)
data=pd.read_csv('./new_metadata_real.csv')

train=data_split['train']
val=data_split['val']
test=data_split['test']

test_sample={}
for i in test:
    print(i)
    for j in os.listdir('Games/'):
        if i in j:
            st=int(data[data['gameid']==int(i)]['new_st'])
            dur=int(data[data['gameid']==int(i)]['new_et'])-int(st)
            sample=[0]*(dur+1)
            file=os.listdir('Games/'+j)
            for f in file:
                try:
                    index=int(f[:-4])-st
                    img = image.load_img('Games/'+j+'/'+f, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x /=255
                    x=np.expand_dims(x, axis=0)
                    feature=fe_model(x)
                    feature_l=(list(np.array(feature).reshape(128)))
                    sample[int(index)]=feature_l
                except:
                    print(i,f)

            test_sample[i]=sample

#                 except:
#                     print(i, f)